<a href="https://colab.research.google.com/github/2SEHI/Factory-Anomaly-Analysis/blob/main/factory_Anomaly_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 연속형 기술통계량
- 평균
- 중위수
- 최빈값
- 산포도
- 분포도 : 왜도, 첨도
- 분산
- 범위

In [7]:
# matplotlib의 colab용 한글 폰트설정
# 실행후 런타임 다시 시작하기
!sudo apt-get install -y fonts-nanum
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-nanum is already the newest version (20170925-1).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.


In [8]:
# 공통 모듈 임포트
import numpy as np
import pandas as pd
import os
from scipy import stats

# 깔끔한 그래프 출력을 위해
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
mpl.rcParams['axes.unicode_minus'] = False

# 그림을 저장할 위치
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "Factory_Anomaly_Detection"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("그림 저장:", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)
    
plt.rc('font', family='NanumBarunGothic') 

import warnings
warnings.filterwarnings(action='ignore')

np.random.seed(42)

In [9]:
fact_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/factory_glass_2016.csv', encoding='cp949')
fact_data.head()

,dat...date.name.,LIFT.OUT.ROL.MTR.M1.SPD.1WSI40101,LIFT.OUT.ROL.MTR.M2.SPD.1WSI40102,LIFT.OUT.ROL.MTR.M3.SPD.1WSI40103,LEHR.DRV.MTR..M4.1.SPD.1WSI40104,LEHR.DRV.MTR..M4.2.SPD.1WSI40105,LEHR.DRV.MTR..M5.1.SPD.1WSI40106,LEHR.DRV.MTR..M5.2.SPD.1WSI40107,LEHR.DRV.MTR..M6.1.SPD.1WSI40108,LEHR.DRV.MTR..M6.2.SPD.1WSI40109,LEHR.DRV.MTR..M7.1.SPD.1WSI40110,LEHR.DRV.MTR..M7.2.SPD.1WSI40111,LEHR.DRV.MTR..M8.1.SPD.1WSI40112,LEHR.DRV.MTR..M8.2.SPD.1WSI40113,LEHR.DRV.MTR..M9.1.SPD.1WSI40114,LEHR.DRV.MTR..M9.2.SPD.1WSI40115,DRIVE.MOTOR.M10.1.SPEED.1WSI40116,LEHR.DRV.MTR..M10.2.SPD.1WSI40117,LEHR.DRV.MTR..M11.1.SPD.1WSI40118,DRIVE.MOTOR.M11.2.SPEED.1WSI40119,LEHR.DRV.MTR..M12.1.SPD.1WSI40120,LEHR.DRV.MTR..M12.2.SPD.1WSI40121,LEHR.DRV.MTR..M13.1.SPD.1WSI40122,LEHR.DRV.MTR..M13.2.SPD.1WSI40123,LEHR.DRV.MTR..M14.1.SPD.1WSI40124,LEHR.DRV.MTR..M14.2.SPD.1WSI40125,LEHR.DRV.MTR..M15.1.SPD.1WSI40126,LEHR.DRV.MTR..M15.2.SPD.1WSI40127,LEHR.DRV.MTR..M16.1.SPD.1WSI40128,DRIVE.MOTOR.M16.2.SPEED.1WSI40129,LEHR.DRV.MTR..M17.1.SPD.1WSI40130,LEHR.DRV.MTR..M17.2.SPD.1WSI40131,LEHR.DRV.MTR..M18.1.SPD.1WSI40132,LEHR.DRV.MTR..M18.2.SPD.1WSI40133,DRIVE.MOTOR.M19.1.SPEED.1WSI40134,LEHR.DRV.MTR..M19.2.SPD.1WSI40135,X1WOS45101_PV,X1WOS45102_PV,X1WOS45103_PV,X1WOS45104_PV,...,S_C1_C2_L_L,S_C1_C2_L_R,S_C2_C3_U_L,S_C2_C3_U_R,S_C2_C3_L_L,S_C2_C3_L_R,S_C3_C4_U_L,S_C3_C4_U_R,S_C3_C4_L_L,S_C3_C4_L_R,S_AB1_AB11_L,S_AB1_AB11_R,S_AB11_AB7_L,S_AB11_AB7_R,S_AB7_C4_L,S_AB7_C4_R,S_C4_RET_L,S_C4_RET_R,S_RET_F_L,S_RET_F_R,STD_U_L,STD_U_R,STD_L_L,STD_L_R,STD_GLS_L,STD_GLS_R,투입.LEFT.,투입.RIGHT.,ROLLDOWN.폐기.LEFT.,진공패드.폐기.LEFT.,ROLLDOWN.폐기.RIGHT.,진공패드.폐기.RIGHT.,폐기율...,ROLLDOWN.진공패드.폐기율..LEFT.,ROLLDOWN.진공패드.폐기율..RIGHT.,ROLLDOWN.진공패드.폐기율..전체.,L.RD,L.Vac,R.RD,R.Vac
0,2016-01-01 07,-1.772682,-1.713219,-1.734851,-1.590051,-1.636293,-1.575213,-1.636293,-1.548119,-1.824581,-1.548119,-1.8189,-1.548119,-1.779407,-1.548119,-1.779407,-1.548119,-1.806012,-1.548119,-1.775051,-1.548119,-1.776634,-1.548119,-1.766604,-1.548119,-1.775051,-1.638207,-1.728206,-1.674911,-1.641818,-1.662151,-1.636293,-1.622314,-1.653428,-1.548119,-1.807954,-1.73,-1.315444,-1.620631,-1.517655,...,-0.195736,0.706941,1.477203,0.410847,0.491620,0.942180,2.776941,2.492437,2.060406,2.612283,1.726404,1.461145,-1.192642,-1.134471,2.599429,2.690433,-0.365598,-0.659319,-1.963403,-1.621407,0.523950,1.951349,2.248106,2.767676,3.012706,3.069007,80,74,2,5,8,3,11.69,8.75,14.86,11.69,0.025,0.063,0.108,0.041
1,2016-01-01 08,-1.772682,-1.713219,-1.734851,-1.590051,-1.636293,-1.575213,-1.636293,-1.548119,-1.824581,-1.548119,-1.8189,-1.548119,-1.779407,-1.548119,-1.779407,-1.548119,-1.806012,-1.548119,-1.775051,-1.548119,-1.776634,-1.548119,-1.766604,-1.548119,-1.775051,-1.638207,-1.728206,-1.674911,-1.641818,-1.662151,-1.636293,-1.622314,-1.653428,-1.548119,-1.807954,-1.73,-1.315444,-1.620631,-1.517655,...,-0.500742,0.822657,1.495204,0.397591,0.587871,0.938186,2.888919,2.573187,2.161149,2.664726,1.682081,1.414518,-1.109125,-1.042269,2.588703,2.654307,-0.348979,-0.560103,-1.978936,-1.810022,0.566521,2.010125,2.364418,2.842630,3.022998,3.069274,92,75,0,2,10,0,7.19,2.17,13.33,7.19,0.000,0.022,0.133,0.000
2,2016-01-01 09,-1.772682,-1.713219,-1.734851,-1.590051,-1.636293,-1.575213,-1.636293,-1.548119,-1.824581,-1.548119,-1.8189,-1.548119,-1.779407,-1.548119,-1.779407,-1.548119,-1.806012,-1.548119,-1.775051,-1.548119,-1.776634,-1.548119,-1.766604,-1.548119,-1.775051,-1.638207,-1.728206,-1.674911,-1.641818,-1.662151,-1.636293,-1.622314,-1.653428,-1.548119,-1.807954,-1.73,-1.315444,-1.620631,-1.517655,...,-0.164556,0.962210,1.552127,0.460702,0.397216,0.843443,2.868644,2.486441,2.183298,2.638130,1.632955,1.359022,-0.986580,-0.928583,2.550175,2.606185,-0.456071,-0.644073,-1.878611,-1.742395,0.545936,2.002820,2.397034,2.817728,2.936967,2.969952,64,66,1,4,0,0,3.85,7.81,0.00,3.85,0.016,0.063,0.000,0.000
3,2016-01-01 10,-1.772682,-1.713219,-1.734851,-1.590051,-1.636293,-1.575213,-1.636293,-1.548119,-1.824581,-1.548119,-1.8189,-1.548119,-1.779407,-1.548119,

In [10]:
fact_data.describe()

,LIFT.OUT.ROL.MTR.M1.SPD.1WSI40101,LIFT.OUT.ROL.MTR.M2.SPD.1WSI40102,LIFT.OUT.ROL.MTR.M3.SPD.1WSI40103,LEHR.DRV.MTR..M4.1.SPD.1WSI40104,LEHR.DRV.MTR..M4.2.SPD.1WSI40105,LEHR.DRV.MTR..M5.1.SPD.1WSI40106,LEHR.DRV.MTR..M5.2.SPD.1WSI40107,LEHR.DRV.MTR..M6.1.SPD.1WSI40108,LEHR.DRV.MTR..M6.2.SPD.1WSI40109,LEHR.DRV.MTR..M7.1.SPD.1WSI40110,LEHR.DRV.MTR..M7.2.SPD.1WSI40111,LEHR.DRV.MTR..M8.1.SPD.1WSI40112,LEHR.DRV.MTR..M8.2.SPD.1WSI40113,LEHR.DRV.MTR..M9.1.SPD.1WSI40114,LEHR.DRV.MTR..M9.2.SPD.1WSI40115,DRIVE.MOTOR.M10.1.SPEED.1WSI40116,LEHR.DRV.MTR..M10.2.SPD.1WSI40117,LEHR.DRV.MTR..M11.1.SPD.1WSI40118,DRIVE.MOTOR.M11.2.SPEED.1WSI40119,LEHR.DRV.MTR..M12.1.SPD.1WSI40120,LEHR.DRV.MTR..M12.2.SPD.1WSI40121,LEHR.DRV.MTR..M13.1.SPD.1WSI40122,LEHR.DRV.MTR..M13.2.SPD.1WSI40123,LEHR.DRV.MTR..M14.1.SPD.1WSI40124,LEHR.DRV.MTR..M14.2.SPD.1WSI40125,LEHR.DRV.MTR..M15.1.SPD.1WSI40126,LEHR.DRV.MTR..M15.2.SPD.1WSI40127,LEHR.DRV.MTR..M16.1.SPD.1WSI40128,DRIVE.MOTOR.M16.2.SPEED.1WSI40129,LEHR.DRV.MTR..M17.1.SPD.1WSI40130,LEHR.DRV.MTR..M17.2.SPD.1WSI40131,LEHR.DRV.MTR..M18.1.SPD.1WSI40132,LEHR.DRV.MTR..M18.2.SPD.1WSI40133,DRIVE.MOTOR.M19.1.SPEED.1WSI40134,LEHR.DRV.MTR..M19.2.SPD.1WSI40135,X1WOS45101_PV,X1WOS45102_PV,X1WOS45103_PV,X1WOS45104_PV,X1WOS45105_PV,...,S_C1_C2_L_L,S_C1_C2_L_R,S_C2_C3_U_L,S_C2_C3_U_R,S_C2_C3_L_L,S_C2_C3_L_R,S_C3_C4_U_L,S_C3_C4_U_R,S_C3_C4_L_L,S_C3_C4_L_R,S_AB1_AB11_L,S_AB1_AB11_R,S_AB11_AB7_L,S_AB11_AB7_R,S_AB7_C4_L,S_AB7_C4_R,S_C4_RET_L,S_C4_RET_R,S_RET_F_L,S_RET_F_R,STD_U_L,STD_U_R,STD_L_L,STD_L_R,STD_GLS_L,STD_GLS_R,투입.LEFT.,투입.RIGHT.,ROLLDOWN.폐기.LEFT.,진공패드.폐기.LEFT.,ROLLDOWN.폐기.RIGHT.,진공패드.폐기.RIGHT.,폐기율...,ROLLDOWN.진공패드.폐기율..LEFT.,ROLLDOWN.진공패드.폐기율..RIGHT.,ROLLDOWN.진공패드.폐기율..전체.,L.RD,L.Vac,R.RD,R.Vac
count,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,...,8144.000000,8144.000000,8145.000000,8145.000000,8144.000000,8144.000000,8145.000000,8145.000000,8144.000000,8144.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000,8145.000000
mean,0.000728,0.000692,0.000705,0.000628,0.000737,0.000618,0.000737,0.000601,0.000738,0.000601,0.000730,0.000601,0.000701,0.000601,0.000701,0.000601,0.000717,0.000601,0.000698,0.000601,0.000698,0.000601,0.000691,0.000601,0.000698,0.000659,0.000661,0.000677,0.000743,0.000672,0.000737,0.000650,0.000617,0.000601,0.000722,-0.000184,-0.001187,-0.000790,-0.000837,-0.000456,...,-0.000529,-0.001111,-0.000501,0.001043,-0.000364,0.000402,0.000242,0.000237,0.000213,0.000425,-0.000673,-0.000881,0.000358,0.000480,0.000159,0.000037,-0.001295,-0.001154,0.000231,-0.000139,0.000351,0.000007,0.000248,0.000409,-0.000434,-0.000518,93.602210,93.680663,1.562799,1.754451,2.217802,0.786004,3.574095,3.720192,3.356119,3.574095,0.018049,0.019174,0.024859,0.008718
std,1.000121,1.000151,1.000140,1.000200,1.000113,1.000207,1.000113,1.000219,1.000113,1.000219,1.000119,1.000219,1.000144,1.000219,1.000144,1.000219,1.000131,1.000219,1.000146,1.000219,1.000146,1.000219,1.000151,1.000219,1.000146,1.000177,1.000175,1.000162,1.000108,1.000167,1.000113,1.000184,1.000208,1.000219,1.000126,1.000407,0.999604,1.000064,1.000020,1.000309,...,1.000248,0.999685,1.000283,0.999794,1.000335,1.000321,1.000386,1.000377,1.000399,1.000305,1.000166,0.999977,1.000354,1.000295,1.000414,1.000428,0.999453,0.999653,1.000398,1.000417,1.0003

### 결측치 확인 
- 전부 Na인 컬럼이 7개 존재하며 1개의 결측치를 가진 컬럼들이 100개 이상 존재함.

In [24]:
x = fact_data.isnull().sum()
print(x.index)
x = pd.DataFrame(x, index=x.index, columns=['NaN의 개수'])
x
np.select([x > 0 ], [x > 0], default=0)

Index(['dat...date.name.', 'LIFT.OUT.ROL.MTR.M1.SPD.1WSI40101',
       'LIFT.OUT.ROL.MTR.M2.SPD.1WSI40102',
       'LIFT.OUT.ROL.MTR.M3.SPD.1WSI40103', 'LEHR.DRV.MTR..M4.1.SPD.1WSI40104',
       'LEHR.DRV.MTR..M4.2.SPD.1WSI40105', 'LEHR.DRV.MTR..M5.1.SPD.1WSI40106',
       'LEHR.DRV.MTR..M5.2.SPD.1WSI40107', 'LEHR.DRV.MTR..M6.1.SPD.1WSI40108',
       'LEHR.DRV.MTR..M6.2.SPD.1WSI40109',
       ...
       'ROLLDOWN.폐기.RIGHT.', '진공패드.폐기.RIGHT.', '폐기율...',
       'ROLLDOWN.진공패드.폐기율..LEFT.', 'ROLLDOWN.진공패드.폐기율..RIGHT.',
       'ROLLDOWN.진공패드.폐기율..전체.', 'L.RD', 'L.Vac', 'R.RD', 'R.Vac'],
      dtype='object', length=841)


,NaN의 개수
dat...date.name.,0
LIFT.OUT.ROL.MTR.M1.SPD.1WSI40101,0
LIFT.OUT.ROL.MTR.M2.SPD.1WSI40102,0
LIFT.OUT.ROL.MTR.M3.SPD.1WSI40103,0
LEHR.DRV.MTR..M4.1.SPD.1WSI40104,0
...,...
ROLLDOWN.진공패드.폐기율..전체.,0
L.RD,0
L.Vac,0
R.RD,0


In [25]:
fact_data.isnull().sum().sort_values(ascending=False).head(10)

UPPER.RET..Z218.TMP.1TIC42601.PV         8145
ROOF.HEATING.Zone78.1JIC31178.PV         8145
LEHR.DRV.MTR..M19.2.CUR.1WII40135.PV     8145
ROOF.HEATING.Zone76.1JIC31176.PV         8145
ROOF.HEATING.Zone52.1JIC31152.PV         8145
DCS_Offset_RPM_M14.1WOS45114_PV          8145
DCS_Offset_RPM_M13.1WOS45113_PV          8145
ATM.FLOW.SECT.10.N2.H2.1FIC33418.PV.1       1
DB.HEAT.TOP.4.1.2.TMP.CT.1JI40304.PV        1
DB.HEAT.TOP.6.1.2.TMP.CT.1JI40306.PV        1
dtype: int64

In [26]:
fact_data = fact_data.dropna(how='all', axis=1)
fact_data.isnull().sum().sort_values(ascending=False).head(100)

SEC11.N2.H2.9..1FIC33426.PV.1           1
top_6                                   1
DB.HEAT.TOP.3.1.2.TMP.CT.1JI40303.PV    1
DB.HEAT.TOP.4.1.2.TMP.CT.1JI40304.PV    1
DB.HEAT.TOP.5.1.2.TMP.CT.1JI40305.PV    1
                                       ..
CASHING.HOT.AIR.TMP.1TIC33604.PV        1
X1WOS45107_PV.1                         1
DCS_Offset_RPM_M16.1WOS45116_PV.1       1
DCS_Offset_RPM_M17.1WOS45117_PV.1       1
S_C3_C4_L_R                             1
Length: 100, dtype: int64

### 결측치가 존재하는 행만 출력

In [27]:
fact_data[fact_data.isnull().any(axis=1)]

,dat...date.name.,LIFT.OUT.ROL.MTR.M1.SPD.1WSI40101,LIFT.OUT.ROL.MTR.M2.SPD.1WSI40102,LIFT.OUT.ROL.MTR.M3.SPD.1WSI40103,LEHR.DRV.MTR..M4.1.SPD.1WSI40104,LEHR.DRV.MTR..M4.2.SPD.1WSI40105,LEHR.DRV.MTR..M5.1.SPD.1WSI40106,LEHR.DRV.MTR..M5.2.SPD.1WSI40107,LEHR.DRV.MTR..M6.1.SPD.1WSI40108,LEHR.DRV.MTR..M6.2.SPD.1WSI40109,LEHR.DRV.MTR..M7.1.SPD.1WSI40110,LEHR.DRV.MTR..M7.2.SPD.1WSI40111,LEHR.DRV.MTR..M8.1.SPD.1WSI40112,LEHR.DRV.MTR..M8.2.SPD.1WSI40113,LEHR.DRV.MTR..M9.1.SPD.1WSI40114,LEHR.DRV.MTR..M9.2.SPD.1WSI40115,DRIVE.MOTOR.M10.1.SPEED.1WSI40116,LEHR.DRV.MTR..M10.2.SPD.1WSI40117,LEHR.DRV.MTR..M11.1.SPD.1WSI40118,DRIVE.MOTOR.M11.2.SPEED.1WSI40119,LEHR.DRV.MTR..M12.1.SPD.1WSI40120,LEHR.DRV.MTR..M12.2.SPD.1WSI40121,LEHR.DRV.MTR..M13.1.SPD.1WSI40122,LEHR.DRV.MTR..M13.2.SPD.1WSI40123,LEHR.DRV.MTR..M14.1.SPD.1WSI40124,LEHR.DRV.MTR..M14.2.SPD.1WSI40125,LEHR.DRV.MTR..M15.1.SPD.1WSI40126,LEHR.DRV.MTR..M15.2.SPD.1WSI40127,LEHR.DRV.MTR..M16.1.SPD.1WSI40128,DRIVE.MOTOR.M16.2.SPEED.1WSI40129,LEHR.DRV.MTR..M17.1.SPD.1WSI40130,LEHR.DRV.MTR..M17.2.SPD.1WSI40131,LEHR.DRV.MTR..M18.1.SPD.1WSI40132,LEHR.DRV.MTR..M18.2.SPD.1WSI40133,DRIVE.MOTOR.M19.1.SPEED.1WSI40134,LEHR.DRV.MTR..M19.2.SPD.1WSI40135,X1WOS45101_PV,X1WOS45102_PV,X1WOS45103_PV,X1WOS45104_PV,...,S_C1_C2_L_L,S_C1_C2_L_R,S_C2_C3_U_L,S_C2_C3_U_R,S_C2_C3_L_L,S_C2_C3_L_R,S_C3_C4_U_L,S_C3_C4_U_R,S_C3_C4_L_L,S_C3_C4_L_R,S_AB1_AB11_L,S_AB1_AB11_R,S_AB11_AB7_L,S_AB11_AB7_R,S_AB7_C4_L,S_AB7_C4_R,S_C4_RET_L,S_C4_RET_R,S_RET_F_L,S_RET_F_R,STD_U_L,STD_U_R,STD_L_L,STD_L_R,STD_GLS_L,STD_GLS_R,투입.LEFT.,투입.RIGHT.,ROLLDOWN.폐기.LEFT.,진공패드.폐기.LEFT.,ROLLDOWN.폐기.RIGHT.,진공패드.폐기.RIGHT.,폐기율...,ROLLDOWN.진공패드.폐기율..LEFT.,ROLLDOWN.진공패드.폐기율..RIGHT.,ROLLDOWN.진공패드.폐기율..전체.,L.RD,L.Vac,R.RD,R.Vac
3754,2016-06-24 04,1.467846,1.465863,1.466053,1.418393,1.477109,1.419455,1.477109,1.421954,1.396569,1.421954,1.412374,1.421954,1.433805,1.421954,1.433805,1.421954,1.434570,1.421954,1.433707,1.421954,1.438408,1.421954,1.440539,1.421954,1.433707,1.411201,1.469038,1.428242,1.468160,1.418962,1.477109,1.410172,1.466652,1.421954,1.580416,0.684064,0.511284,0.552619,0.552069,...,1.166185,1.199438,-0.545404,0.089552,-1.084166,-0.330890,-0.415981,-0.716057,0.131328,-0.410615,-0.627784,-0.591798,1.315342,1.280796,-1.268809,-1.297407,0.293999,0.076699,0.164519,0.818587,-0.383247,-0.620188,0.039335,-0.485325,-1.160946,-1.395816,82,93,2,2,0,0,2.29,4.88,0.00,2.29,0.024,0.024,0.000,0.0
6310,2016-10-13 09,-0.846817,-0.804910,-0.820307,-0.730496,-0.857942,-0.719593,-0.857942,-0.699527,-0.858236,-0.699527,-0.849518,-0.699527,-0.815443,-0.699527,-0.815443,-0.699527,-0.833838,-0.699527,-0.812423,-0.699527,-0.812122,-0.699527,-0.804461,-0.699527,-0.812423,-0.766948,-0.769033,-0.788296,-0.864323,-0.781833,-0.857942,-0.755890,-0.873408,-0.699527,-0.839848,0.464603,0.815738,0.750187,0.552069,...,-0.059873,-0.502049,-0.784489,-0.868543,-0.150358,-0.294429,-0.038451,0.070333,-0.078548,0.050499,0.526827,0.547936,-0.496371,-0.425395,-0.013030,-0.011655,0.653207,0.620476,-0.700757,-0.644046,0.360002,0.394778,-0.142186,0.017076,0.422987,0.453458,82,74,6,5,12,0,14.74,13.41,16.22,14.74,0.073,0.061,0.162,0.0
8044,2016-12-27 17,-0.846817,-0.804910,-0.820307,-0.730496,-0.857942,-0.719593,-0.857942,-0.699527,-0.858236,-0.699527,-0.849518,-0.699527,-0.815443,-0.699527,-0.815443,-0.699527,-0.833838,-0.699527,-0.812423,-0.699527,-0.812122,-0.699527,-0.804461,-0.699527,-0.812423,-0.766948,-0.769033,-0.788296,-0.864323,-0.781833,-0.857942,-0.755890,-0.717404,-0.699527,-0.839848,0.025683,-1.315444,-0.830358,0.404232,...,NaN,NaN,0.335541,-1.305447,NaN,NaN,-0.496747,-0.467255,NaN,NaN,0.009370,0.204032,0.031374,-0.194762,-1.189177,-0.902694,2.042967,1.706486,0.402644,1.121082,-0.827081,-0.429306,-10.387414,-11.711583,7.669278,7.949552,55,53,0,0,3,0,2.78,0.00,5.66,2.78,0.000,0.000,0.057,0.0


## 결측치가 존재하는 행의 NaN값 출력과 삭제

In [28]:
print(fact_data.iloc[3754].isnull().sum())
print(fact_data.iloc[6310].isnull().sum())
print(fact_data.iloc[8044].isnull().sum())

fact_data = fact_data.dropna(how='any')
fact_data

1
39
87


,dat...date.name.,LIFT.OUT.ROL.MTR.M1.SPD.1WSI40101,LIFT.OUT.ROL.MTR.M2.SPD.1WSI40102,LIFT.OUT.ROL.MTR.M3.SPD.1WSI40103,LEHR.DRV.MTR..M4.1.SPD.1WSI40104,LEHR.DRV.MTR..M4.2.SPD.1WSI40105,LEHR.DRV.MTR..M5.1.SPD.1WSI40106,LEHR.DRV.MTR..M5.2.SPD.1WSI40107,LEHR.DRV.MTR..M6.1.SPD.1WSI40108,LEHR.DRV.MTR..M6.2.SPD.1WSI40109,LEHR.DRV.MTR..M7.1.SPD.1WSI40110,LEHR.DRV.MTR..M7.2.SPD.1WSI40111,LEHR.DRV.MTR..M8.1.SPD.1WSI40112,LEHR.DRV.MTR..M8.2.SPD.1WSI40113,LEHR.DRV.MTR..M9.1.SPD.1WSI40114,LEHR.DRV.MTR..M9.2.SPD.1WSI40115,DRIVE.MOTOR.M10.1.SPEED.1WSI40116,LEHR.DRV.MTR..M10.2.SPD.1WSI40117,LEHR.DRV.MTR..M11.1.SPD.1WSI40118,DRIVE.MOTOR.M11.2.SPEED.1WSI40119,LEHR.DRV.MTR..M12.1.SPD.1WSI40120,LEHR.DRV.MTR..M12.2.SPD.1WSI40121,LEHR.DRV.MTR..M13.1.SPD.1WSI40122,LEHR.DRV.MTR..M13.2.SPD.1WSI40123,LEHR.DRV.MTR..M14.1.SPD.1WSI40124,LEHR.DRV.MTR..M14.2.SPD.1WSI40125,LEHR.DRV.MTR..M15.1.SPD.1WSI40126,LEHR.DRV.MTR..M15.2.SPD.1WSI40127,LEHR.DRV.MTR..M16.1.SPD.1WSI40128,DRIVE.MOTOR.M16.2.SPEED.1WSI40129,LEHR.DRV.MTR..M17.1.SPD.1WSI40130,LEHR.DRV.MTR..M17.2.SPD.1WSI40131,LEHR.DRV.MTR..M18.1.SPD.1WSI40132,LEHR.DRV.MTR..M18.2.SPD.1WSI40133,DRIVE.MOTOR.M19.1.SPEED.1WSI40134,LEHR.DRV.MTR..M19.2.SPD.1WSI40135,X1WOS45101_PV,X1WOS45102_PV,X1WOS45103_PV,X1WOS45104_PV,...,S_C1_C2_L_L,S_C1_C2_L_R,S_C2_C3_U_L,S_C2_C3_U_R,S_C2_C3_L_L,S_C2_C3_L_R,S_C3_C4_U_L,S_C3_C4_U_R,S_C3_C4_L_L,S_C3_C4_L_R,S_AB1_AB11_L,S_AB1_AB11_R,S_AB11_AB7_L,S_AB11_AB7_R,S_AB7_C4_L,S_AB7_C4_R,S_C4_RET_L,S_C4_RET_R,S_RET_F_L,S_RET_F_R,STD_U_L,STD_U_R,STD_L_L,STD_L_R,STD_GLS_L,STD_GLS_R,투입.LEFT.,투입.RIGHT.,ROLLDOWN.폐기.LEFT.,진공패드.폐기.LEFT.,ROLLDOWN.폐기.RIGHT.,진공패드.폐기.RIGHT.,폐기율...,ROLLDOWN.진공패드.폐기율..LEFT.,ROLLDOWN.진공패드.폐기율..RIGHT.,ROLLDOWN.진공패드.폐기율..전체.,L.RD,L.Vac,R.RD,R.Vac
0,2016-01-01 07,-1.772682,-1.713219,-1.734851,-1.590051,-1.636293,-1.575213,-1.636293,-1.548119,-1.824581,-1.548119,-1.818900,-1.548119,-1.779407,-1.548119,-1.779407,-1.548119,-1.806012,-1.548119,-1.775051,-1.548119,-1.776634,-1.548119,-1.766604,-1.548119,-1.775051,-1.638207,-1.728206,-1.674911,-1.641818,-1.662151,-1.636293,-1.622314,-1.653428,-1.548119,-1.807954,-1.730000,-1.315444,-1.620631,-1.517655,...,-0.195736,0.706941,1.477203,0.410847,0.491620,0.942180,2.776941,2.492437,2.060406,2.612283,1.726404,1.461145,-1.192642,-1.134471,2.599429,2.690433,-0.365598,-0.659319,-1.963403,-1.621407,0.523950,1.951349,2.248106,2.767676,3.012706,3.069007,80,74,2,5,8,3,11.69,8.75,14.86,11.69,0.025,0.063,0.108,0.041
1,2016-01-01 08,-1.772682,-1.713219,-1.734851,-1.590051,-1.636293,-1.575213,-1.636293,-1.548119,-1.824581,-1.548119,-1.818900,-1.548119,-1.779407,-1.548119,-1.779407,-1.548119,-1.806012,-1.548119,-1.775051,-1.548119,-1.776634,-1.548119,-1.766604,-1.548119,-1.775051,-1.638207,-1.728206,-1.674911,-1.641818,-1.662151,-1.636293,-1.622314,-1.653428,-1.548119,-1.807954,-1.730000,-1.315444,-1.620631,-1.517655,...,-0.500742,0.822657,1.495204,0.397591,0.587871,0.938186,2.888919,2.573187,2.161149,2.664726,1.682081,1.414518,-1.109125,-1.042269,2.588703,2.654307,-0.348979,-0.560103,-1.978936,-1.810022,0.566521,2.010125,2.364418,2.842630,3.022998,3.069274,92,75,0,2,10,0,7.19,2.17,13.33,7.19,0.000,0.022,0.133,0.000
2,2016-01-01 09,-1.772682,-1.713219,-1.734851,-1.590051,-1.636293,-1.575213,-1.636293,-1.548119,-1.824581,-1.548119,-1.818900,-1.548119,-1.779407,-1.548119,-1.779407,-1.548119,-1.806012,-1.548119,-1.775051,-1.548119,-1.776634,-1.548119,-1.766604,-1.548119,-1.775051,-1.638207,-1.728206,-1.674911,-1.641818,-1.662151,-1.636293,-1.622314,-1.653428,-1.548119,-1.807954,-1.730000,-1.315444,-1.620631,-1.517655,...,-0.164556,0.962210,1.552127,0.460702,0.397216,0.843443,2.868644,2.486441,2.183298,2.638130,1.632955,1.359022,-0.986580,-0.928583,2.550175,2.606185,-0.456071,-0.644073,-1.878611,-1.742395,0.545936,2.002820,2.397034,2.817728,2.936967,2.969952,64,66,1,4,0,0,3.85,7.81,0.00,3.85,0.016,0.063,0.000,0.000
3,2016-01-01 10,-1.772682,-1.713219,-1.734851,-1.590051,-1.636293,-1.575213,-1.636293,-1.548119,-1.824581,-1.548119,-1.818900,-1.548119,

In [29]:
# date와 폐기율 컬럼 제외 
drop_columns = ['dat...date.name.', '투입.LEFT.','투입.RIGHT.','ROLLDOWN.폐기.LEFT.','진공패드.폐기.LEFT.','ROLLDOWN.폐기.RIGHT.','진공패드.폐기.RIGHT.','폐기율...','ROLLDOWN.진공패드.폐기율..LEFT.','ROLLDOWN.진공패드.폐기율..RIGHT.','ROLLDOWN.진공패드.폐기율..전체.','L.RD','L.Vac','R.RD','R.Vac',]
columns = np.array(fact_data.drop(columns=drop_columns).columns)

In [38]:
drop_columns = ['dat...date.name.', '투입.LEFT.','투입.RIGHT.','ROLLDOWN.폐기.LEFT.','진공패드.폐기.LEFT.','ROLLDOWN.폐기.RIGHT.','진공패드.폐기.RIGHT.','폐기율...','ROLLDOWN.진공패드.폐기율..LEFT.','ROLLDOWN.진공패드.폐기율..RIGHT.','ROLLDOWN.진공패드.폐기율..전체.','L.RD','L.Vac','R.RD','R.Vac',]
columns_all = np.array(fact_data.drop(columns=drop_columns).columns)
# column_1 = fact_data.iloc[:, :50].columns
column_1 = fact_data.iloc[:, :50].columns

array(['LIFT.OUT.ROL.MTR.M1.SPD.1WSI40101',
       'LIFT.OUT.ROL.MTR.M2.SPD.1WSI40102',
       'LIFT.OUT.ROL.MTR.M3.SPD.1WSI40103',
       'LEHR.DRV.MTR..M4.1.SPD.1WSI40104',
       'LEHR.DRV.MTR..M4.2.SPD.1WSI40105',
       'LEHR.DRV.MTR..M5.1.SPD.1WSI40106',
       'LEHR.DRV.MTR..M5.2.SPD.1WSI40107',
       'LEHR.DRV.MTR..M6.1.SPD.1WSI40108',
       'LEHR.DRV.MTR..M6.2.SPD.1WSI40109',
       'LEHR.DRV.MTR..M7.1.SPD.1WSI40110',
       'LEHR.DRV.MTR..M7.2.SPD.1WSI40111',
       'LEHR.DRV.MTR..M8.1.SPD.1WSI40112',
       'LEHR.DRV.MTR..M8.2.SPD.1WSI40113',
       'LEHR.DRV.MTR..M9.1.SPD.1WSI40114',
       'LEHR.DRV.MTR..M9.2.SPD.1WSI40115',
       'DRIVE.MOTOR.M10.1.SPEED.1WSI40116',
       'LEHR.DRV.MTR..M10.2.SPD.1WSI40117',
       'LEHR.DRV.MTR..M11.1.SPD.1WSI40118',
       'DRIVE.MOTOR.M11.2.SPEED.1WSI40119',
       'LEHR.DRV.MTR..M12.1.SPD.1WSI40120',
       'LEHR.DRV.MTR..M12.2.SPD.1WSI40121',
       'LEHR.DRV.MTR..M13.1.SPD.1WSI40122',
       'LEHR.DRV.MTR..M13.2.SPD.1WSI40123',


## 상관관계가 가장 낮은 컬럼만 몇개 출력해보기

In [ ]:
# date와 폐기율 컬럼 제외 
# drop_columns = ['dat...date.name.', '투입.LEFT.','투입.RIGHT.','ROLLDOWN.폐기.LEFT.','진공패드.폐기.LEFT.','ROLLDOWN.폐기.RIGHT.','진공패드.폐기.RIGHT.','폐기율...','ROLLDOWN.진공패드.폐기율..LEFT.','ROLLDOWN.진공패드.폐기율..RIGHT.','ROLLDOWN.진공패드.폐기율..전체.','L.RD','L.Vac','R.RD','R.Vac',]
# columns = np.array(fact_data.drop(columns=drop_columns).columns)

# 상관계수가 0.6이하의 독립변수를 담는 list
columns_isol = []
# 상관계수를 담는 list
# corr_list = []

for i in range(len(columns_all)):
    for j in range(i+1, len(columns_all)):
        c = fact_data[[columns_all[i],columns_all[j]]].corr().abs()
        s = c.unstack()
        so = s.sort_values(kind="quicksort", ascending=False)
        if so[3] < 0.6 :
            columns_isol.append(columns[i])
            # corr_list.append(so[3])

In [ ]:
# 독립변수로만 이루어진 DataFrame
fact_data_isol = fact_data[columns_isol]
fact_data_isol

In [ ]:
# 평균, 
fact_data_isol.describe()

## 중심경향성

In [ ]:
fact_data.describe()

In [ ]:
## 중앙값 median()
fact_data = fact_data[columns]
print(fact_data.median())

# 최빈값 scipy의 mode()
print(stats.mode(fact_data['LIFT.OUT.ROL.MTR.M1.SPD.1WSI40101'])[0])


# 범위
np.max(fact_data['LIFT.OUT.ROL.MTR.M1.SPD.1WSI40101']) - np.min(fact_data['LIFT.OUT.ROL.MTR.M1.SPD.1WSI40101'])

## 분산도

In [ ]:
# 분산 var()
print(np.var(fact_data['LIFT.OUT.ROL.MTR.M1.SPD.1WSI40101']))


## 표준 편차

In [ ]:
fact_data

In [ ]:
fact_data['LIFT.OUT.ROL.MTR.M1.SPD.1WSI40101'].value_counts().plot(kind='scatter')

In [ ]:
c = fact_data[['폐기율...','ROLLDOWN.진공패드.폐기율..전체.']].corr().abs()
c

In [ ]:
plt.title('시간별 폐기율')
plt.plot(fact_data['폐기율...'])
plt.ylim([0, fact_data['폐기율...'].max()])
plt.show()

In [ ]:
plt.title('일별 폐기율')
plt.boxplot(fact_data['폐기율...'])
plt.show()

In [ ]:
fact_data['폐기율...'].describe()

In [ ]:
index = fact_data['폐기율...'].sort_values(ascending=False).head(10).index
index

In [ ]:
index = fact_data['폐기율...'].sort_values(ascending=False).head(10).index
fact_data.iloc[index]['dat...date.name.']

In [ ]:
so[3]

In [ ]:
list[3]